In [90]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn import *
import seaborn as sns
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import warnings
import mglearn
import random
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.feature_selection import SelectPercentile
import os
import glob
from sklearn.svm import SVC
from sklearn.metrics.scorer import make_scorer
import tqdm

files = glob.glob("./*.csv")

# 説明変数となる行列X, 被説明変数となるy2を作成
base = 100 
day_ago = 30
pre_day=1
reset =True

#出来高やボリンジャーバンドなど指標の数
num_sihyou = 16

# すべてのCSVファイルから得微量作成
for file in files:
    temp = pd.read_csv(file, header=0, encoding='cp932')
    temp = temp[['日付','始値', '高値','安値','終値','5日平均','25日平均','75日平均','出来高']]
    temp= temp.iloc[::-1]#上下反対に
    temp2 = np.array(temp)
    
    # 前日比を出すためにbase日後からのデータを取得
    temp3 = np.zeros((len(temp2)-base, num_sihyou))
    temp3[0:len(temp3), 0] = temp2[base:len(temp2), 4].astype(np.float) / temp2[base-1:len(temp2)-1, 4].astype(np.float)
    temp3[0:len(temp3), 1] = temp2[base:len(temp2), 1].astype(np.float)/ temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 2] = temp2[base:len(temp2), 2].astype(np.float)/ temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 3] = temp2[base:len(temp2), 3].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 4] = temp2[base:len(temp2), 5].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 5] = temp2[base:len(temp2), 6].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 6] = temp2[base:len(temp2), 7].astype(np.float) / temp2[base:len(temp2), 4].astype(np.float)
    temp3[0:len(temp3), 7] = temp2[base:len(temp2), 8].astype(np.float) / temp2[base-1:len(temp2)-1, 8].astype(np.float)
    
     # 一目均衡表を追加 (9,26, 52) 
    para1 =9
    para2 = 26
    para3 = 52
    temp2_2 = np.c_[temp2, np.zeros((len(temp2), 3))]
    
    # 転換線 = （過去(para1)日間の高値 + 安値） ÷ 2
    for i in range(para1, len(temp2)):
        tmp_high =temp2[i-para1+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para1+1:i+1,3].astype(np.float)
        temp2_2[i, 9] = (np.max(tmp_high) + np.min(tmp_low))/2
        
    temp3[0:len(temp3), 8] = temp2_2[base:len(temp2), 9]/temp2[base:len(temp2), 4]

    # 基準線 = （過去(para2)日間の高値 + 安値） ÷ 2
    for i in range(para2, len(temp2)):
        tmp_high =temp2[i-para2+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para2+1:i+1,3].astype(np.float)
        temp2_2[i, 10] = (np.max(tmp_high) + np.min(tmp_low)) / 2
    temp3[0:len(temp3), 9] = temp2_2[base:len(temp2), 10]/temp2[base:len(temp2), 4]
        
    # 先行スパン1 = ｛ （転換値+基準値） ÷ 2 ｝を(para2)日先にずらしたもの
    temp3[0:len(temp3), 10] = (temp2_2[base-para2:len(temp2)-para2, 9] + temp2_2[base-para2:len(temp2)-para2, 10]) /2 /temp2[base:len(temp2), 4]

    # 先行スパン2 = ｛ （過去(para3)日間の高値+安値） ÷ 2 ｝を(para2)日先にずらしたもの
    for i in range(para3, len(temp2)-1):
        tmp_high =temp2[i-para3+1:i+1,2].astype(np.float)
        tmp_low =temp2[i-para3+1:i+1,3].astype(np.float)
        temp2_2[i, 11] = (np.max(tmp_high) + np.min(tmp_low)) / 2
    temp3[0:len(temp3), 11] = temp2_2[base-para2:len(temp2)-para2, 11]/temp2[base:len(temp2), 4]

    # 25日ボリンジャーバンド（±1, 2シグマ）を追加
    parab = 25
    for i in range(base, len(temp2)):
        tmp25 = temp2[i-parab+1:i+1,4].astype(np.float)
        temp3[i-base,12] = np.mean(tmp25) + 1.0* np.std(tmp25) 
        temp3[i-base,13] = np.mean(tmp25) - 1.0* np.std(tmp25) 
        temp3[i-base,14] = np.mean(tmp25) + 2.0* np.std(tmp25) 
        temp3[i-base,15] = np.mean(tmp25) - 2.0* np.std(tmp25)
     
    # tempX : 現在の企業のデータ
    tempX = np.zeros((len(temp3), day_ago*num_sihyou))
    
    # 日にちごとに横向きに（day_ago）分並べる
    for s in range(0, num_sihyou): 
        for i in range(0, day_ago):
            tempX[i:len(temp3), day_ago*s+i] = temp3[0:len(temp3)-i,s]
             
    # Xに追加
    # X : すべての企業のデータ
    if reset:
        X = tempX[day_ago:len(temp3)-pre_day]
        y = tempX[day_ago+pre_day:len(temp3)+pre_day,0]
        reset = False
    else:
        X = np.concatenate((X, tempX[day_ago:len(temp3)-pre_day]), axis=0)
        y = np.concatenate((y, tempX[day_ago+pre_day:len(temp3)+pre_day,0]), axis=0)

up_rate =1.000

# y : pre_day後の終値/当日終値

#それぞれの範囲でデータ数をカウント
a=b=0
yc = np.zeros(len(y))
for i in range(0, len(yc)):
    if y[i]>=up_rate:
        yc[i]=1
        a+=1
    else :
        yc[i] = -1
        b+=1
print("-1",b)
print("1",a)

-1 102413
1 114989


In [91]:
print(len(X))
print(len(y))
print(len(X_train))
print(len(X_test))
print(len(y_train))
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,test_size=0.2)
yc_train=np.zeros(len(y_train))
yc_test=np.zeros(len(y_test))
for i in range(0, len(y_train)):
    if y_train[i]>up_rate:
        yc_train[i]=1
        a+=1
    else :
        yc_train[i] = -1
        b+=1
print("1",a)
print("-1",b)

c=d=0
for i in range(0, len(y_test)):
    if y_test[i]>up_rate:
        yc_test[i]=1
        c+=1
    else :
        yc_test[i] = -1
        d+=1
print("1",c)
print("-1",d)

swich=True
def calc_score(y_train_or_test: np.array, y_pred: np.array):
    #print("y_train shape:",y_train.shape)
    score=0.000
    if swich:
        for i in range (0,(len(y_train_or_test))):
            score+=float(y_pred[i])*(y_train[i]-1.00000)
            #print(y_pred[i],y_train[i])
        score=score/float(len(y_train_or_test))
    elif swich==False:
        for i in range (0,(len(y_train_or_test))):
            score+=y_pred[i].astype(np.float)*(y_test[i]-1.000000)
            print(float(y_pred[i]),y_test[i],yc_test[i])
        score=score/float(len(y_train_or_test))
    return score

pipe = Pipeline([('scaler', StandardScaler()), ('classifier', MLPClassifier(max_iter=200, alpha=0.1))])

param_grid = {'classifier__hidden_layer_sizes': [(100,), (500,),(1000,)]}

sontoku = make_scorer(calc_score, greater_is_better=True)
    
grid = GridSearchCV(pipe, param_grid=param_grid, n_jobs=1, cv=2 ,return_train_score=False, scoring=sontoku)
grid.fit(X_train, yc_train)

print(grid.cv_results_['mean_test_score'])
print("Best parameters: ", grid.best_params_)
print("grid best score, ", grid.best_score_)
swich=False
print("Test set score: {:.2f}".format(grid.score(X_test, yc_test)))
pred=grid.predict(X_test)
print("score",calc_score(y_test,pred))

217402
217402
172313
43079
172313
1 195502
-1 195821
1 20225
-1 23256
[0.00063295 0.00051827 0.00061614]
Best parameters:  {'classifier__hidden_layer_sizes': (100,)}
grid best score,  0.0006329549838270956
-1.0 0.9704433497536946 -1.0
-1.0 1.000592768227623 1.0
-1.0 0.9829787234042553 -1.0
-1.0 1.0 -1.0
-1.0 0.9863481228668942 -1.0
-1.0 0.9000805801772764 -1.0
-1.0 1.0091074681238617 1.0
-1.0 0.9893689986282579 -1.0
-1.0 1.0137111517367459 1.0
-1.0 0.987012987012987 -1.0
-1.0 0.9493392070484582 -1.0
-1.0 0.9848484848484849 -1.0
1.0 1.009090909090909 1.0
-1.0 0.9942528735632183 -1.0
-1.0 0.9944444444444445 -1.0
-1.0 1.0104895104895104 1.0
-1.0 1.002683504340963 1.0
-1.0 1.030261348005502 1.0
-1.0 0.9911560328490209 -1.0
-1.0 0.986763732627399 -1.0
-1.0 1.0 -1.0
-1.0 0.9773371104815864 -1.0
-1.0 0.9885057471264368 -1.0
-1.0 1.0497297297297297 1.0
1.0 1.0 -1.0
-1.0 0.9708179202630497 -1.0
1.0 0.9947643979057592 -1.0
-1.0 0.9827760891590679 -1.0
-1.0 0.9777227722772277 -1.0
-1.0 0.97974683

-1.0 1.0044345898004434 1.0
-1.0 1.0 -1.0
-1.0 1.108695652173913 1.0
-1.0 0.9802816901408451 -1.0
-1.0 0.9949567723342939 -1.0
-1.0 1.009453781512605 1.0
1.0 0.9935064935064936 -1.0
-1.0 1.0082644628099173 1.0
-1.0 1.0029282576866765 1.0
-1.0 0.9887591240875913 -1.0
-1.0 1.0 -1.0
-1.0 0.9779005524861878 -1.0
-1.0 0.9375 -1.0
-1.0 1.0055555555555555 1.0
-1.0 1.0140237324703345 1.0
1.0 0.9437834109329194 -1.0
1.0 1.0 -1.0
-1.0 1.0 -1.0
-1.0 1.020979020979021 1.0
-1.0 1.0846153846153845 1.0
-1.0 1.0 -1.0
-1.0 1.0689655172413792 1.0
-1.0 0.9742173112338858 -1.0
1.0 1.0043668122270741 1.0
-1.0 1.0008237232289952 1.0
-1.0 1.0057142857142858 1.0
-1.0 0.9414634146341463 -1.0
-1.0 0.9954819277108434 -1.0
-1.0 1.0108108108108107 1.0
-1.0 1.001150417026172 1.0
1.0 1.0085763293310464 1.0
-1.0 1.0434782608695652 1.0
-1.0 0.9933155080213903 -1.0
-1.0 1.002415458937198 1.0
1.0 0.9757281553398058 -1.0
-1.0 1.032768978700164 1.0
-1.0 0.9761092150170648 -1.0
-1.0 0.7894736842105263 -1.0
1.0 1.0220820189

-1.0 0.9946120689655172 -1.0
1.0 1.0682730923694779 1.0
-1.0 1.0092133238837704 1.0
-1.0 0.9803921568627451 -1.0
-1.0 1.0061099796334012 1.0
-1.0 0.9919028340080972 -1.0
1.0 0.9688346883468835 -1.0
-1.0 1.0 -1.0
-1.0 1.0338028169014084 1.0
1.0 1.0188679245283019 1.0
-1.0 1.0102915951972555 1.0
-1.0 0.9939759036144579 -1.0
-1.0 1.005181347150259 1.0
-1.0 1.0 -1.0
-1.0 1.0243161094224924 1.0
-1.0 1.0088105726872247 1.0
-1.0 0.9924357034795764 -1.0
-1.0 0.9770700636942675 -1.0
-1.0 1.0096308186195826 1.0
-1.0 0.9859154929577465 -1.0
-1.0 1.05 1.0
-1.0 0.9805510534846029 -1.0
-1.0 1.0031347962382444 1.0
-1.0 0.9786628733997155 -1.0
-1.0 0.9642513934268692 -1.0
-1.0 0.9743589743589743 -1.0
-1.0 1.0105691056910568 1.0
-1.0 0.9950310559006211 -1.0
-1.0 1.0357142857142858 1.0
-1.0 1.0043478260869565 1.0
1.0 0.977639751552795 -1.0
1.0 0.9973821989528796 -1.0
1.0 1.0534521158129175 1.0
-1.0 0.9803921568627451 -1.0
-1.0 0.9800995024875622 -1.0
-1.0 0.9924091090691171 -1.0
-1.0 0.9769820971867008 

-1.0 1.0083333333333333 1.0
-1.0 1.017515923566879 1.0
-1.0 0.9850209706411025 -1.0
-1.0 1.0172413793103448 1.0
-1.0 1.0226119058606369 1.0
-1.0 1.0029641185647427 1.0
-1.0 1.018032786885246 1.0
-1.0 0.9960369881109643 -1.0
-1.0 0.9912280701754386 -1.0
-1.0 1.0291929021179165 1.0
1.0 0.9920477137176938 -1.0
-1.0 1.0 -1.0
-1.0 1.0344827586206897 1.0
-1.0 1.006578947368421 1.0
-1.0 1.004172461752434 1.0
-1.0 0.9812734082397003 -1.0
1.0 0.9896907216494846 -1.0
-1.0 0.9928707224334601 -1.0
1.0 0.9716312056737588 -1.0
-1.0 0.9919028340080972 -1.0
1.0 0.9909502262443439 -1.0
-1.0 1.0129155197967394 1.0
-1.0 1.002020202020202 1.0
-1.0 0.9983498349834984 -1.0
-1.0 0.9899396378269618 -1.0
-1.0 1.0208333333333333 1.0
-1.0 1.0084033613445378 1.0
-1.0 1.0171730515191546 1.0
-1.0 1.0317399617590821 1.0
1.0 0.984081041968162 -1.0
1.0 1.1030927835051547 1.0
1.0 0.9344841114162417 -1.0
-1.0 1.0 -1.0
-1.0 0.9907834101382489 -1.0
-1.0 1.0 -1.0
-1.0 1.0 -1.0
-1.0 1.0045045045045045 1.0
-1.0 0.99554234769

-1.0 0.9676470588235294 -1.0
-1.0 1.0060541849553504 1.0
1.0 1.0207253886010363 1.0
-1.0 0.9735449735449735 -1.0
-1.0 1.0168539325842696 1.0
-1.0 1.0054972513743128 1.0
-1.0 0.9877300613496932 -1.0
-1.0 1.0122282608695652 1.0
-1.0 1.04 1.0
-1.0 1.0190114068441065 1.0
-1.0 0.978448275862069 -1.0
-1.0 1.0080645161290323 1.0
-1.0 0.9811320754716981 -1.0
1.0 1.0273244781783681 1.0
-1.0 1.0104166666666667 1.0
-1.0 0.9922600619195047 -1.0
1.0 1.0027322404371584 1.0
-1.0 1.0 -1.0
-1.0 1.0148093298778231 1.0
1.0 1.0287015945330296 1.0
-1.0 1.007415254237288 1.0
-1.0 0.9693877551020408 -1.0
-1.0 1.0052910052910053 1.0
-1.0 1.0017123287671232 1.0
-1.0 1.0294117647058822 1.0
-1.0 0.9536423841059603 -1.0
-1.0 1.0138852501964895 1.0
-1.0 1.0083272990586531 1.0
1.0 0.9935613994259561 -1.0
-1.0 0.9790382244143033 -1.0
-1.0 0.9631043256997456 -1.0
-1.0 1.010677808727948 1.0
-1.0 0.9625390218522373 -1.0
-1.0 0.9753761969904241 -1.0
-1.0 0.9979402677651905 -1.0
-1.0 1.0176730486008836 1.0
1.0 0.98245614

...

-1.0 1.0159010600706713 1.0
1.0 1.04 1.0
-1.0 1.0 -1.0
-1.0 1.01018922852984 1.0
1.0 0.9661458333333334 -1.0
1.0 0.9914754098360655 -1.0
-1.0 0.9888059701492538 -1.0
-1.0 0.9926442121564073 -1.0
-1.0 1.0 -1.0
-1.0 1.0032051282051282 1.0
-1.0 1.0180995475113122 1.0
1.0 1.061881188118812 1.0
-1.0 1.0066889632107023 1.0
-1.0 0.9924528301886792 -1.0
-1.0 1.005281690140845 1.0
1.0 1.0141784820683903 1.0
-1.0 1.0798319327731092 1.0
1.0 1.0118953211736716 1.0
1.0 1.0297029702970297 1.0
-1.0 0.9823594266813671 -1.0
-1.0 1.0597014925373134 1.0
-1.0 0.97610513739546 -1.0
-1.0 0.9591576260370134 -1.0
-1.0 1.0198863636363635 1.0
-1.0 1.0693324733956788 1.0
-1.0 1.0 -1.0
1.0 0.9809312638580931 -1.0
-1.0 1.0 -1.0
1.0 1.0144230769230769 1.0
-1.0 0.9934640522875817 -1.0
1.0 0.9942196531791907 -1.0
-1.0 1.035897435897436 1.0
-1.0 0.9621621621621622 -1.0
-1.0 0.9932634730538922 -1.0
-1.0 0.9766666666666667 -1.0
-1.0 0.989804587935429 -1.0
-1.0 1.0 -1.0
-1.0 1.0459770114942528 1.0
-1.0 1.0 -1.0
-1.0 0.99

1.0 0.9978768577494692 -1.0
1.0 0.9900332225913622 -1.0
-1.0 1.0235655737704918 1.0
-1.0 1.054773082942097 1.0
-1.0 1.0214477211796247 1.0
-1.0 1.0035696073431923 1.0
-1.0 0.9780876494023905 -1.0
-1.0 0.9841269841269841 -1.0
-1.0 0.9919012726571539 -1.0
-1.0 1.0091205211726384 1.0
-1.0 1.0123456790123457 1.0
1.0 1.0270485282418456 1.0
-1.0 1.0707964601769913 1.0
-1.0 1.013550135501355 1.0
1.0 0.9741176470588235 -1.0
-1.0 1.0039370078740157 1.0
1.0 1.0018050541516246 1.0
-1.0 1.0124095139607032 1.0
-1.0 1.0055248618784531 1.0
-1.0 0.9809046932020707 -1.0
-1.0 1.0217391304347827 1.0
-1.0 1.0540832882639264 1.0
-1.0 1.0031746031746032 1.0
-1.0 1.0055248618784531 1.0
-1.0 1.01 1.0
1.0 1.0588235294117647 1.0
-1.0 1.0135135135135136 1.0
1.0 0.98834628190899 -1.0
-1.0 0.9955207166853304 -1.0
-1.0 1.0115942028985507 1.0
-1.0 0.9874213836477987 -1.0
-1.0 0.9902571474205398 -1.0
1.0 1.0012738853503185 1.0
-1.0 0.9961832061068703 -1.0
-1.0 0.9626436781609196 -1.0
1.0 0.9831460674157303 -1.0
-1.0 

-1.0 0.98989898989899 -1.0
-1.0 0.9967503159415056 -1.0
-1.0 1.008714596949891 1.0
-1.0 0.9853680841335163 -1.0
1.0 0.9936305732484076 -1.0
1.0 0.9420289855072463 -1.0
-1.0 1.0134089736977823 1.0
-1.0 0.9713375796178344 -1.0
-1.0 1.0020325203252032 1.0
-1.0 1.0092348284960422 1.0
-1.0 0.9818956336528222 -1.0
-1.0 0.9960513326752222 -1.0
-1.0 1.010989010989011 1.0
-1.0 1.0112359550561798 1.0
-1.0 1.0 -1.0
-1.0 0.968503937007874 -1.0
1.0 0.9943502824858758 -1.0
-1.0 1.0076579876140375 1.0
-1.0 1.0243362831858407 1.0
1.0 1.0163170163170163 1.0
1.0 0.9901840490797545 -1.0
-1.0 0.9804017638412543 -1.0
-1.0 1.0200364298724955 1.0
-1.0 0.9880952380952381 -1.0
-1.0 0.9900398406374502 -1.0
1.0 1.0122950819672132 1.0
-1.0 1.0108303249097472 1.0
-1.0 1.0167597765363128 1.0
-1.0 0.9716796875 -1.0
-1.0 0.9779527559055118 -1.0
-1.0 0.9966640502354788 -1.0
-1.0 0.9795197740112994 -1.0
-1.0 0.9776833156216791 -1.0
-1.0 0.9928973465558831 -1.0
-1.0 1.0107991360691144 1.0
-1.0 1.0 -1.0
1.0 1.00333333333

-1.0 1.0578512396694215 1.0
-1.0 0.9711538461538461 -1.0
1.0 0.9904076738609112 -1.0
-1.0 1.0111461619348054 1.0
-1.0 1.015267175572519 1.0
1.0 1.0100755667506298 1.0
-1.0 1.0 -1.0
-1.0 0.9968014329580348 -1.0
-1.0 1.0018709073900842 1.0
1.0 0.9839743589743589 -1.0
-1.0 1.002016129032258 1.0
-1.0 0.9902067464635473 -1.0
-1.0 0.9964071856287425 -1.0
-1.0 1.0022918258212377 1.0
1.0 1.0292044310171198 1.0
-1.0 1.0 -1.0
-1.0 0.9787234042553191 -1.0
-1.0 1.0235294117647058 1.0
-1.0 0.9965753424657534 -1.0
-1.0 0.985981308411215 -1.0
-1.0 0.984354628422425 -1.0
-1.0 1.0140562248995983 1.0
-1.0 0.9818360333824251 -1.0
-1.0 0.9869608826479438 -1.0
-1.0 0.9967213114754099 -1.0
-1.0 0.9796845124282982 -1.0
-1.0 1.0 -1.0
-1.0 1.043879907621247 1.0
-1.0 1.0 -1.0
-1.0 0.9893758300132802 -1.0
1.0 0.9658399625643426 -1.0
-1.0 0.9615384615384616 -1.0
-1.0 0.9984350547730829 -1.0
-1.0 1.004524886877828 1.0
-1.0 1.0193055555555555 1.0
-1.0 1.0157407407407408 1.0
-1.0 0.974009900990099 -1.0
-1.0 1.01 1.0

-1.0 1.0127388535031847 1.0
-1.0 0.9783333333333334 -1.0
-1.0 0.9898568019093079 -1.0
-1.0 0.9977924944812362 -1.0
-1.0 0.9879140555058191 -1.0
-1.0 1.0421792618629173 1.0
-1.0 1.035778175313059 1.0
1.0 0.9975429975429976 -1.0
-1.0 1.0037313432835822 1.0
1.0 1.0301204819277108 1.0
-1.0 1.0317919075144508 1.0
1.0 1.1040582726326742 1.0
-1.0 1.0250521920668059 1.0
-1.0 0.9948717948717949 -1.0
-1.0 0.9875 -1.0
-1.0 1.0142314990512333 1.0
-1.0 0.983030303030303 -1.0
-1.0 1.0247524752475248 1.0
-1.0 0.9785867237687366 -1.0
1.0 1.0017985611510791 1.0
1.0 1.0751879699248121 1.0
1.0 1.017094017094017 1.0
-1.0 0.9951100244498777 -1.0
-1.0 0.9991721854304636 -1.0
-1.0 1.04149377593361 1.0
-1.0 0.9900990099009901 -1.0
1.0 1.007462686567164 1.0
-1.0 0.9827586206896551 -1.0
-1.0 0.9984177215189873 -1.0
-1.0 1.0089887640449438 1.0
-1.0 0.9200603318250377 -1.0
-1.0 1.020997375328084 1.0
-1.0 1.039855072463768 1.0
-1.0 1.0239520958083832 1.0
-1.0 1.0066489361702127 1.0
-1.0 1.003875968992248 1.0
-1.0 

In [82]:
pred=grid.predict(X_test)
print("score",calc_score(y_test,pred))

1.0 1.0047169811320755 1.0
-1.0 0.9901743264659271 -1.0
-1.0 1.018425460636516 1.0
-1.0 0.9708179202630497 -1.0
1.0 1.009185303514377 1.0
-1.0 0.9794776119402985 -1.0
-1.0 1.1153402537485582 1.0
1.0 0.9932432432432432 -1.0
-1.0 0.9833333333333333 -1.0
1.0 1.0095238095238095 1.0
1.0 1.0468253968253969 1.0
-1.0 0.9859649122807017 -1.0
-1.0 0.9832402234636871 -1.0
-1.0 0.9911383776414451 -1.0
-1.0 0.9913444893248702 -1.0
-1.0 0.977645305514158 -1.0
-1.0 0.9995147986414362 -1.0
-1.0 0.9852941176470589 -1.0
-1.0 0.995249406175772 -1.0
1.0 0.9943502824858758 -1.0
-1.0 0.9937106918238994 -1.0
-1.0 1.0177635887360839 1.0
score 0.003609554503999221
